In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd


base_url = 'https://www.abs.gov.au/statistics/standards/australian-statistical-geography-standard-asgs-edition-3/jul2021-jun2026/access-and-downloads/allocation-files'


df_MB = pd.read_excel(f'{base_url}/MB_2021_AUST.xlsx')
# 3m 30s


df_LGA = pd.read_excel(f'{base_url}/LGA_2023_AUST.xlsx')
# 2m


df_SAL = pd.read_excel(f'{base_url}/SAL_2021_AUST.xlsx')
# 3m 30s


df_POA = pd.read_excel(f'{base_url}/POA_2021_AUST.xlsx')
# 2m


df = pd.merge(df_MB, df_LGA, on='MB_CODE_2021', how='left', suffixes=('', '_LGA'))
df = pd.merge(df, df_SAL, on='MB_CODE_2021', how='left', suffixes=('', '_SAL'))
df = pd.merge(df, df_POA, on='MB_CODE_2021', how='left', suffixes=('', '_POA'))
# 10s

# Drop duplicate columns
df.drop(columns=['STATE_CODE_2021_LGA', 'STATE_NAME_2021_LGA', 'AUS_CODE_2021_LGA', 'AUS_NAME_2021_LGA', 'AREA_ALBERS_SQKM_LGA', 'ASGS_LOCI_URI_2021_LGA', 'STATE_CODE_2021_SAL', 'STATE_NAME_2021_SAL', 'AUS_CODE_2021_SAL', 'AUS_NAME_2021_SAL', 'AREA_ALBERS_SQKM_SAL', 'ASGS_LOCI_URI_2021_SAL', 'AUS_CODE_2021_POA', 'AUS_NAME_2021_POA', 'AREA_ALBERS_SQKM_POA', 'ASGS_LOCI_URI_2021_POA'], inplace=True)


gdf : gpd.GeoDataFrame = gpd.read_file('local/MB_2021_AUST_SHP_GDA2020/MB_2021_AUST_GDA2020.shp')
# 8m

gdf : gpd.GeoDataFrame = gdf[['MB_CODE21', 'geometry']].rename(columns={'MB_CODE21': 'MB_CODE_2021'}).merge(df, on='MB_CODE_2021', how='left')
# 10s

gdf_suburbs = gdf[['SAL_CODE_2021', 'POA_CODE_2021', 'geometry']].dissolve(by=['SAL_CODE_2021', 'POA_CODE_2021']).reset_index()
# 3m 30s


df_suburbs = df[['SAL_CODE_2021', 'POA_CODE_2021', 'STATE_CODE_2021', 'AUS_CODE_2021']].drop_duplicates().reset_index(drop=True)

# gdf_suburbs_area = df.groupby(['SAL_CODE_2021', 'POA_CODE_2021', 'STATE_CODE_2021', 'AUS_CODE_2021'])['AREA_ALBERS_SQKM'].sum().reset_index()

C:\Users\An\AppData\Roaming\Python\Python312\site-packages\geopandas\geodataframe.py:1815: FutureWarning: `unary_union` returned None due to all-None GeoSeries. In future, `unary_union` will return 'GEOMETRYCOLLECTION EMPTY' instead.
  merged_geom = block.unary_union


In [2]:
df_suburbs_info = df[[
    'SAL_CODE_2021',
    'POA_CODE_2021',
    'STATE_CODE_2021',
    'AUS_CODE_2021',
    'SAL_NAME_2021',
    'POA_NAME_2021',
    'STATE_NAME_2021',
    'AUS_NAME_2021',
]].drop_duplicates().reset_index(drop=True)

In [3]:
df_suburbs_area = df.groupby(['SAL_CODE_2021','POA_CODE_2021'])['AREA_ALBERS_SQKM'].sum().reset_index()

In [4]:
df_suburbs_area

,SAL_CODE_2021,POA_CODE_2021,AREA_ALBERS_SQKM
0,10001,2850,82.7639
1,10002,2176,4.9788
2,10003,2046,1.0180
3,10004,2795,2.9775
4,10005,2795,127.1701
...,...,...,...
15535,90004,2899,38.6510
15536,90005,6799,5.9276
15537,99494,9494,0.0000
15538,99797,9797,0.0000


In [5]:
gdf_suburbs : gpd.GeoDataFrame = gdf_suburbs.merge(df_suburbs_area, on=['SAL_CODE_2021', 'POA_CODE_2021'], how='left').merge(df_suburbs_info, on=['SAL_CODE_2021', 'POA_CODE_2021'], how='left')

In [6]:
gdf_suburbs.rename(columns={
    'SAL_CODE_2021' : 'suburb_id',
    'POA_CODE_2021' : 'postcode_id',
    'STATE_CODE_2021' : 'state_id',
    'AUS_CODE_2021' : 'country_id',
    'SAL_NAME_2021' : 'suburb',
    'POA_NAME_2021' : 'postcode',
    'STATE_NAME_2021' : 'state_name',
    'AUS_NAME_2021' : 'country_name',
    'AREA_ALBERS_SQKM' : 'area_sqkm',
}, inplace=True)

gdf_suburbs['state'] = gdf_suburbs['state_id'].map({
    '1' : 'NSW',
    '2' : 'VIC',
    '3' : 'QLD',
    '4' : 'SA',
    '5' : 'WA',
    '6' : 'TAS',
    '7' : 'NT',
    '8' : 'ACT',
    '9' : 'Other Territories',
    'Z' : 'Outside Australia',
})

In [7]:
gdf_suburbs

,suburb_id,postcode_id,geometry,area_sqkm,state_id,country_id,suburb,postcode,state_name,country_name,state
0,10001,2850,"POLYGON ((149.74733 -32.91034, 149.74741 -32.9...",82.7639,1,AUS,Aarons Pass,2850,New South Wales,Australia,NSW
1,10002,2176,"POLYGON ((150.86479 -33.88339, 150.86463 -33.8...",4.9788,1,AUS,Abbotsbury,2176,New South Wales,Australia,NSW
2,10003,2046,"POLYGON ((151.12703 -33.85554, 151.12693 -33.8...",1.0180,1,AUS,Abbotsford (NSW),2046,New South Wales,Australia,NSW
3,10004,2795,"POLYGON ((149.54550 -33.39044, 149.54361 -33.3...",2.9775,1,AUS,Abercrombie,2795,New South Wales,Australia,NSW
4,10005,2795,"POLYGON ((149.25562 -33.96535, 149.25563 -33.9...",127.1701,1,AUS,Abercrombie River,2795,New South Wales,Australia,NSW
...,...,...,...,...,...,...,...,...,...,...,...
15535,90004,2899,"MULTIPOLYGON (((167.94108 -29.06217, 167.94131...",38.6510,9,AUS,Norfolk Island,2899,Other Territories,Australia,Other Territories
15536,90005,6799,"MULTIPOLYGON (((96.83046 -12.17626, 96.83048 -...",5.9276,9,AUS,West Island,6799,Other Territories,Australia,Other Territories
15537,99494,9494,None,0.0000,9,AUS,No usual address (OT),No usual address (Aust.),Other Territories,Australia,Other Territories
15538,99797,9797,"MULTIPOLYGON (((167.97281 -29.01566, 167.97384...",0.0000,9,AUS,Migratory - Offshore - Shipping (OT),Migratory - Offshore - Shipping (Aust.),Other Territories,Australia,Other Territories


In [8]:
# Split the suburbs into separate states, and for each state, split into 5 geojson files
for state_id in states.keys():
    gdfs = gdf_suburbs[gdf_suburbs['state_id'] == state_id]
    # Sort by postcode
    gdfs.sort_values('postcode', inplace=True)
    gdfs.reset_index(drop=True, inplace=True)
    # Split into 5 geojson files
    n = len(gdfs)
    n_per_file = n // 5
    for i in range(5):
        gdfs[i*n_per_file:(i+1)*n_per_file].to_file(f'local/suburbs-{states[state_id]}-{i}.geojson', driver='GeoJSON')

NameError: name 'states' is not defined

In [9]:
states = {
    '1' : 'NSW',
    '2' : 'VIC',
    '3' : 'QLD',
    '4' : 'SA',
    '5' : 'WA',
    '6' : 'TAS',
    '7' : 'NT',
    '8' : 'ACT',
    '9' : 'Other Territories',
    'Z' : 'Outside Australia',
}
gdf_suburbs['state'] = gdf_suburbs['state_id'].map(states)

In [10]:
# Split the suburbs into separate states, and for each state, split into 5 geojson files
for state_id in states.keys():
    gdfs = gdf_suburbs[gdf_suburbs['state_id'] == state_id]
    # Sort by postcode
    gdfs.sort_values('postcode', inplace=True)
    gdfs.reset_index(drop=True, inplace=True)
    # Split into 5 geojson files
    n = len(gdfs)
    n_per_file = n // 5
    for i in range(5):
        gdfs[i*n_per_file:(i+1)*n_per_file].to_file(f'local/suburbs-{states[state_id]}-{i}.geojson', driver='GeoJSON')

<ipython-input-10-9ddc70620e2a>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gdfs.sort_values('postcode', inplace=True)


KeyboardInterrupt: 

In [11]:
# Split the suburbs into separate states, and for each state, split into 5 geojson files
for state_id in states.keys():
    gdfs = gdf_suburbs[gdf_suburbs['state_id'] == state_id].sort_values('postcode').reset_index(drop=True)
    # Split into 5 geojson files
    n = len(gdfs)
    n_per_file = n // 5
    for i in range(5):
        gdfs[i*n_per_file:(i+1)*n_per_file].to_file(f'local/suburbs-{states[state_id]}-{i}.geojson', driver='GeoJSON')
        print(f'local/suburbs-{states[state_id]}-{i}.geojson')

local/suburbs-NSW-0.geojson
local/suburbs-NSW-1.geojson
local/suburbs-NSW-2.geojson
local/suburbs-NSW-3.geojson
local/suburbs-NSW-4.geojson
local/suburbs-VIC-0.geojson
local/suburbs-VIC-1.geojson
local/suburbs-VIC-2.geojson
local/suburbs-VIC-3.geojson
local/suburbs-VIC-4.geojson
local/suburbs-QLD-0.geojson
local/suburbs-QLD-1.geojson
local/suburbs-QLD-2.geojson
local/suburbs-QLD-3.geojson
local/suburbs-QLD-4.geojson
local/suburbs-SA-0.geojson
local/suburbs-SA-1.geojson
local/suburbs-SA-2.geojson
local/suburbs-SA-3.geojson
local/suburbs-SA-4.geojson
local/suburbs-WA-0.geojson
local/suburbs-WA-1.geojson
local/suburbs-WA-2.geojson
local/suburbs-WA-3.geojson
local/suburbs-WA-4.geojson
local/suburbs-TAS-0.geojson
local/suburbs-TAS-1.geojson
local/suburbs-TAS-2.geojson
local/suburbs-TAS-3.geojson
local/suburbs-TAS-4.geojson
local/suburbs-NT-0.geojson
local/suburbs-NT-1.geojson
local/suburbs-NT-2.geojson
local/suburbs-NT-3.geojson
local/suburbs-NT-4.geojson
local/suburbs-ACT-0.geojson
local/s

C:\Users\An\AppData\Roaming\Python\Python312\site-packages\geopandas\io\file.py:608: UserWarning: You are attempting to write an empty DataFrame to file. For some drivers, this operation may fail.
  _to_file_fiona(df, filename, driver, schema, crs, mode, **kwargs)
C:\Users\An\AppData\Roaming\Python\Python312\site-packages\geopandas\io\file.py:608: UserWarning: You are attempting to write an empty DataFrame to file. For some drivers, this operation may fail.
  _to_file_fiona(df, filename, driver, schema, crs, mode, **kwargs)
C:\Users\An\AppData\Roaming\Python\Python312\site-packages\geopandas\io\file.py:608: UserWarning: You are attempting to write an empty DataFrame to file. For some drivers, this operation may fail.
  _to_file_fiona(df, filename, driver, schema, crs, mode, **kwargs)
C:\Users\An\AppData\Roaming\Python\Python312\site-packages\geopandas\io\file.py:608: UserWarning: You are attempting to write an empty DataFrame to file. For some drivers, this operation may fail.
  _to_fi